## 载入数据，简单分析

In [1]:
# 这两行的作用是使每个cell的执行局结果可以显示多个
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 下面这三行代码是为了画图可以显示中文
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
import pandas as pd 
import numpy as np
import time 
import datetime
from dateutil.parser import parse
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
from sklearn.preprocessing import robust_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold
%matplotlib inline
#plt.rcParams['figure.figsize'] = (12.0, 6.0) #set default size of plots

C:\Users\Tengfei\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# -*- coding: utf-8 -*-
data_train = pd.read_csv('d_train_20180102.csv',encoding = 'gbk')     # 在读入文件时，如果有中文，则会报错，添加参数encoding = 'gbk'即可
data_test = pd.read_csv('d_test_A_20180102.csv',encoding = 'gbk')     
print("训练集的形状：" + format(data_train.shape))
print("测试集的形状：" + format(data_test.shape))

训练集的形状：(5642, 42)
测试集的形状：(1000, 41)


In [7]:
data_train.head()
data_test.head()

,id,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,...,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,血糖
0,1,男,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,...,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,6.06
1,2,男,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,...,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,5.39
2,3,男,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,...,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,5.59
3,4,女,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,...,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,4.30
4,5,女,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,...,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,5.42


,id,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,...,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%
0,5733,男,54,10/10/2017,23.85,26.69,116.08,34.36,82.75,46.03,...,12.3,241.0,10.8,12.8,0.26,58.4,33.2,7.5,0.6,0.3
1,5734,男,50,10/10/2017,29.75,34.98,90.07,111.43,71.90,44.09,...,12.0,242.0,11.5,14.6,0.28,59.3,29.3,7.7,3.2,0.5
2,5735,男,27,10/10/2017,NaN,NaN,NaN,NaN,NaN,NaN,...,12.1,398.0,8.9,9.9,0.35,50.2,40.1,7.9,1.2,0.6
3,5736,女,53,10/10/2017,17.98,16.63,95.95,23.41,78.16,45.44,...,12.6,247.0,11.6,13.9,0.29,53.7,38.0,7.2,0.7,0.4
4,5739,女,43,10/10/2017,19.12,19.80,76.97,15.70,80.76,46.90,...,12.2,335.0,10.4,11.9,0.35,52.0,39.4,8.0,0.3,0.3


训练集的样本为5642个，特征字段有42个，测试集的样本有1000个，特征字段有41个。测试集的最后一列血糖是需要预测的，被删掉了

In [5]:
# 检查病人的特征数据是否有缺失
data_train.isnull().any()
print('---------------------------------')
data_test.isnull().any()

id             False
性别             False
年龄             False
体检日期           False
*天门冬氨酸氨基转换酶     True
*丙氨酸氨基转换酶       True
*碱性磷酸酶          True
*r-谷氨酰基转换酶      True
*总蛋白            True
白蛋白             True
*球蛋白            True
白球比例            True
甘油三酯            True
总胆固醇            True
高密度脂蛋白胆固醇       True
低密度脂蛋白胆固醇       True
尿素              True
肌酐              True
尿酸              True
乙肝表面抗原          True
乙肝表面抗体          True
乙肝e抗原           True
乙肝e抗体           True
乙肝核心抗体          True
白细胞计数           True
红细胞计数           True
血红蛋白            True
红细胞压积           True
红细胞平均体积         True
红细胞平均血红蛋白量      True
红细胞平均血红蛋白浓度     True
红细胞体积分布宽度       True
血小板计数           True
血小板平均体积         True
血小板体积分布宽度       True
血小板比积           True
中性粒细胞%          True
淋巴细胞%           True
单核细胞%           True
嗜酸细胞%           True
嗜碱细胞%           True
血糖             False
dtype: bool

---------------------------------


id             False
性别             False
年龄             False
体检日期           False
*天门冬氨酸氨基转换酶     True
*丙氨酸氨基转换酶       True
*碱性磷酸酶          True
*r-谷氨酰基转换酶      True
*总蛋白            True
白蛋白             True
*球蛋白            True
白球比例            True
甘油三酯            True
总胆固醇            True
高密度脂蛋白胆固醇       True
低密度脂蛋白胆固醇       True
尿素              True
肌酐              True
尿酸              True
乙肝表面抗原          True
乙肝表面抗体          True
乙肝e抗原           True
乙肝e抗体           True
乙肝核心抗体          True
白细胞计数           True
红细胞计数           True
血红蛋白            True
红细胞压积           True
红细胞平均体积         True
红细胞平均血红蛋白量      True
红细胞平均血红蛋白浓度     True
红细胞体积分布宽度       True
血小板计数           True
血小板平均体积         True
血小板体积分布宽度       True
血小板比积           True
中性粒细胞%          True
淋巴细胞%           True
单核细胞%           True
嗜酸细胞%           True
嗜碱细胞%           True
dtype: bool

根据上面的结果显示，除了病人的id,性别，年龄，体检日期，目标血糖这几个相关的基本信息没有缺失值，其余字段都会有或多或少的病人在某些字段含有缺失值，下面先统计一下这些字段的缺失值的数量可视化

In [ ]:
data_train.isnull().sum()

In [ ]:
print('训练集各特征字段的数据缺失比例（%）')
(data_train.isnull().sum() / data_train.shape[0]) * 100

In [ ]:
print('测试集各特征数据缺失比例（%）')
(data_test.isnull().sum()/(data_test.shape[0])) * 100

In [ ]:
#data_train['体检日期'] = (pd.to_datetime(data_train['体检日期'],format='%d/%m/%Y') - parse('2017-09-15')).dt.days
#data_test['体检日期'] = (pd.to_datetime(data_test['体检日期'],format='%d/%m/%Y') - parse('2017-09-15')).dt.days

In [ ]:

feature_count = pd.Series(data_train.count())     # 将病人特征的未缺失数据统计放在一个Series中
total_count = pd.Series(np.tile(data_train.shape[0], len(feature_count)), index = feature_count.index)# total_count中存放的是全部样本数量
missing_count = total_count - feature_count     # 每个特征的缺失值数量
data_count = pd.DataFrame([feature_count, missing_count]).T      
data_count = data_count.rename(columns = {0:'Not Missing', 1: 'Missing'})     # 修改列名称

# 下面画出每个字段缺失数据与未缺失数据的条形图，由于在一张图中显示太过拥挤，所以用两张图显示
plt.figure(figsize = (12, 10))
data_count[0:24].plot(kind = 'barh', stacked = True)
plt.legend(loc = 'upper right')
plt.show()
plt.figure(figsize = (12, 10))
data_count[24:].plot(kind = 'barh', stacked = True)
plt.legend(loc = 'upper left')
plt.show()

由上面的结果可以看出，训练集和测试集的所有字段可以分为以下几类：
- 缺失值比例极多的字段特征：**乙肝e抗体**，**乙肝e抗原**，**乙肝表面抗体**，**乙肝表面抗原**，**乙肝核心抗体**
- 缺失值比例较少的字段特征：**尿酸**，**肌酐**，**尿素**，**低密度脂蛋白胆固醇**，**高密度脂蛋白胆固醇**，**总胆固醇**，**甘油三脂**，**白球比例**，**球蛋白**，**白蛋白**，**总蛋白**，**r-谷氨酰基转换酶**，**碱性磷酸酶**，**丙氨酸氨基转换酶**，**天东门氨酸氨基转换酶**
- 缺失比例极少的字段特征：**嗜碱细胞%**，**嗜酸细胞%**，**单核细胞%**，**淋巴细胞%**，**中性粒细胞%**，**血小板比积**，**血小板体积分布宽度**，**血小板平均体积**，**血小板计数**，**红细胞体积分布宽度**，**红细胞平均血红蛋白浓度**，**红细胞平均血红蛋白量**，**红细胞平均体积**，**红细胞压积**，**血红蛋白**，**红细胞计数**，**白细胞计数**
- 完全没有缺失数据的字段特征：**体检日期**，**年龄**，**性别**，**id**，**血糖**

In [ ]:
sns.countplot('性别', data = data_train)
plt.title('训练集中的男女性别统计')
plt.xlabel('性别')
plt.ylabel('人数')
plt.show()
print('-----------------------------------------------------')
sns.countplot('性别', data = data_test)
plt.title('测试集中的男女性别统计')
plt.xlabel('性别')
plt.ylabel('人数')
plt.show()

In [ ]:
unknown_sex = data_train.loc[data_train['性别'] == '??'].index
unknown_sex

发现虽然性别这一特征并没有缺失值，但是在572行中有一个性别未知的样本，通过原始数据文件，可以发现，该样本周围的样本都是女性，因此，暂且将该未知性别假设为样本输入时产生的错误，将该样本记为女性，为了下面便于数据处理，将样本中的男性标为1，女性标记为0

In [ ]:
data_train.loc[572, '性别'] = '女'
data_train.loc[data_train['性别']== "男", '性别'] = 1
data_train.loc[data_train['性别']== '女', '性别'] = 0

In [ ]:
data_test.loc[data_test['性别']=='男', '性别'] = 1
data_test.loc[data_test['性别']=='女', '性别'] = 0

In [ ]:
sns.countplot('性别', data = data_train)
classes = ['女', '男']
tick_marks = np.arange(len(classes))
plt.title('训练集中的男女性别统计')
plt.xlabel('性别')
plt.xticks(tick_marks, classes)
plt.ylabel('人数')
plt.show()
print('-----------------------------------------------------')
sns.countplot('性别', data = data_test)
plt.title('测试集中的男女性别统计')
plt.xticks(tick_marks, classes)
plt.xlabel('性别')
plt.ylabel('人数')
plt.show()

先看一下样本中血糖值的分布情况吧，看看有没有异常值

In [ ]:
from scipy import stats
fig = plt.figure( figsize = (6, 6))
sns.distplot(data_train['血糖'], fit = stats.norm)
plt.show()
res = stats.probplot(data_train['血糖'], plot = plt)

注意到训练集中有个样本的血糖值异常地高，血糖值为38.43，该样本的id号为4228,在下面的分析中，将该样本去掉

In [ ]:
data_train.drop(data_train[data_train['id']==4228].index, axis = 0, inplace = True)

将血糖值取对数，看下概率分布图是否会有所改善

In [ ]:
from scipy import stats
fig = plt.figure( figsize = (6, 6))
sns.distplot(np.log(data_train['血糖']), fit = stats.norm)
plt.show()
res = stats.probplot(np.log(data_train['血糖']), plot = plt)

将血糖值取对数之后，概率分布图更加弯曲了（不符合正态分布），所以接下来不对血糖值进行对数变换

In [ ]:
data_train.columns

由于乙肝e抗体，乙肝e抗原，乙肝表面抗体，乙肝表面抗原，乙肝核心抗体这五个特征字段缺失值比例在70%以上，对模型的分析和预测没有什么帮助，所以去掉这几个特征字段；还有id特征字段只是样本的编号，在接下来的分析中不会用到此字段，因此也将其删除。

In [ ]:
#data_train = data_train.drop(columns = 'id')
#data_test = data_test.drop(columns = 'id')
data_train = data_train.drop(columns = ['乙肝e抗原','乙肝e抗体','乙肝表面抗原','乙肝表面抗体','乙肝核心抗体','id', '体检日期'])
data_test = data_test.drop(columns = ['乙肝e抗原','乙肝e抗体','乙肝表面抗原','乙肝表面抗体','乙肝核心抗体','id', '体检日期'])
data_train.columns   # 删除这几列之后的训练集特种字段

In [ ]:
# 对比一下男性与女性血糖值的直方图分布
male = data_train[data_train['性别'] == 1]
female = data_train[data_train['性别'] == 0]
fig, axes = plt.subplots(1, 2, figsize = (12, 6))
male['血糖'].hist(bins = 70, ax = axes[0])
female['血糖'].hist(bins = 70, ax = axes[1])
axes[0].set_title('男性血糖值直方图')
axes[1].set_title('女性血糖值直方图')

样本中的男性血糖较高的人数比女性多

In [ ]:
print('各个特征与血糖的线性相关系数')
print('------------------------------')
data_train.corrwith(data_train['血糖'])   # 看看各个单变量与血糖之间的线性关系
print('------------------------------')
datacorr = data_train.corr()      # 计算各个变量之间的相关系数矩阵
print('\n\n************************各个特征之间的相关系数矩阵*********************')
fig = plt.figure(figsize = (18,18))
sns.heatmap(datacorr, annot = True, fmt = ".2f")    # 用heatmap将相关系数矩阵显示出来
plt.show()

没有哪个特征与血糖之间是线性相关性较高的，中性粒细胞%和淋巴细胞%、*球蛋白和白球比例之间具有高度的负相关性

根据上面的分析，将缺失值超过50%的特征字段进行了删除，下面对剩余缺失字段进行缺失值填补，剩余的缺失字段中，可以分为两类：一类是缺失值比例极少（少于1%）的特征字段，另一类是缺失比例在（20%左右）的特征字段。
- 用[Q1, Q3]之间的随机数填充  
注：Q1是四分之一位数，Q3是四分之三位数

In [ ]:
# 有缺失值的特征字段名称
missing_features = data_train.iloc[:, 2:-1].columns
missing_features
len(missing_features)

### 训练集缺失值处理

In [ ]:
fig, axs = plt.subplots(figsize = (12, 32 * 6), nrows = 32, ncols = 2)
train_info = data_train.describe()     # 用于生成训练集的中值，均值，分位数等信息
for row, cn in  enumerate(missing_features):
    train_cn_Q1 = train_info.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（插值前）
    train_cn_Q3 = train_info.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（插值前）
    
    train_cn_IQR = train_cn_Q3 - train_cn_Q1
    train_cn_maxQ = train_cn_Q3 + (1.5 * train_cn_IQR)
    train_cn_minQ = train_cn_Q1 - (1.5 * train_cn_IQR)
    
    count_nan_cn_train = data_train[cn].isnull().sum()   # 统计训练集特征字段cn的缺失值数量
    # 生成与特征字段在区间 [Q1, Q3]的随机数，用于填补缺失值
    rand_1 = np.random.randint(low = train_cn_minQ, high = train_cn_maxQ + 1, size = count_nan_cn_train)    
    rand_d = np.random.rand(count_nan_cn_train)
    rand_im = rand_1 + rand_d
    sns.distplot(data_train[cn].dropna(), ax = axs[row, 0])
    
    data_train.loc[np.isnan(data_train[cn]), cn] = rand_im    # 在缺失值的位置填充上生成的随机数
    
    sns.distplot(data_train[cn], ax = axs[row, 1])
    axs[row, 0].set_title(str(cn) + '插值前的直方图')
    axs[row, 0].set_xlabel('')
    axs[row, 1].set_title(str(cn) + '插值后的直方图')
    axs[row, 1].set_xlabel('')
plt.show()

In [ ]:
data_train.isnull().any()

### 测试集缺失值处理

In [ ]:
fig, axs = plt.subplots(figsize = (12, 32*6), nrows = 32, ncols = 2)
test_info = data_test.describe()     # 用于生成训练集的中值，均值，分位数等信息
for row, cn in  enumerate(missing_features):
    test_cn_Q1 = test_info.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（插值前）
    test_cn_Q3 = test_info.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（插值前）
    
    test_cn_IQR = test_cn_Q3 - test_cn_Q1
    test_cn_maxQ = test_cn_Q3 + (1.5 * test_cn_IQR)
    test_cn_minQ = test_cn_Q1 - (1.5 * test_cn_IQR)
    
    count_nan_cn_test = data_test[cn].isnull().sum()   # 统计训练集特征字段cn的缺失值数量
    # 生成与特征字段在区间 [Q1, Q3]的随机数，用于填补缺失值
    rand_1 = np.random.randint(low = test_cn_minQ, high = test_cn_maxQ + 1, size = count_nan_cn_test)    
    rand_d = np.random.rand(count_nan_cn_test)
    rand_im = rand_1 + rand_d
    sns.distplot(data_test[cn].dropna(), ax = axs[row, 0])
    
    data_test.loc[np.isnan(data_test[cn]), cn] = rand_im    # 在缺失值的位置填充上生成的随机数
    
    sns.distplot(data_test[cn], ax = axs[row, 1])
    axs[row, 0].set_title(str(cn) + '插值前的直方图')
    axs[row, 0].set_xlabel('')
    axs[row, 1].set_title(str(cn) + '插值后的直方图')
    axs[row, 1].set_xlabel('')
plt.show()

In [ ]:
data_test.isnull().any()

In [ ]:
data_train.to_csv('data_train_A.csv', encoding = 'gbk')
data_test.to_csv('data_test_A.csv', encoding = 'gbk')

### 异常值处理  
注：在处理异常值之后，发现会稍微改变数据原来的分布，根据后面的测试结果来看，处理过异常值之后再用xgboost模型训练，结果并不会有提升（当然，这都是后话了）

In [ ]:
features = data_test.columns  # 特征字段名称
features 

In [ ]:
for i in features:
    fig, ax = plt.subplots(figsize = (6, 4))
    sns.boxplot(data = data_train[i])
    ax.set_title('箱形图：' + str(i))
    plt.show()

In [ ]:
for i in features:
    fig, ax = plt.subplots(figsize = (6, 4))
    sns.boxplot(data = data_test[i])
    ax.set_title('箱形图：' + str(i))
    plt.show()

可以看出，训练集除了性别没有异常值之外，其余特征都有异常值，测试集中，性别和年龄没有异常值，其余特征也是有异常值，下面对异常值进行处理，异常值处理按照如下规则：
- 当一个特征的值大于 Q3 + 1.5IQR 时，异常值用[Q3, Q3 + 1.5IQR]区间的随机数代替
- 当一个特征值小于 Q1 - 1.5IQR 时，异常值用[Q1 - 1.5IQR, Q1]区间的随机数代替

In [ ]:
train_info = data_train.describe()     # 用于生成训练集的中值，均值，分位数等信息
train_info2 = train_info

In [ ]:
for cn in features:
    train_cn_Q1 = train_info.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（处理异常值前）
    train_cn_Q3 = train_info.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（处理异常值前）
    train_cn_IQR = train_cn_Q3 - train_cn_Q1
    train_cn_maxQ = train_cn_Q3 + (1.5 * train_cn_IQR)

    Largeoutlier_cn_count = data_train.loc[data_train[cn] > train_cn_maxQ ,cn].count() # 训练集特征字段cn超过Q3 + 1.5IQR上界的异常值个数
    Largeoutlier_cn_index = data_train.loc[data_train[cn] > train_cn_maxQ, cn].index
    # 生成与特征字段在区间 [Q3, Q3 + 1.5IQR]的随机数，用于填补缺失值
    if (train_cn_maxQ - train_cn_Q3) > 1:
        rand_1max = np.random.randint(low = train_cn_Q3, high = train_cn_maxQ, size = Largeoutlier_cn_count)    
        rand_dmax = np.random.rand(Largeoutlier_cn_count)    # 生成[0, 1)之间的小数
        rand_imax = rand_1max + rand_dmax    
        data_train.loc[Largeoutlier_cn_index, cn] = rand_imax    # 在异常值的位置填充上生成的随机数
    else:    # 有些特征的值范围非常小，小于1，这样的特征异常值，令其等于train_cn_maxQ
        data_train.loc[Largeoutlier_cn_index, cn] = train_cn_maxQ  

for cn in features:
    train_cn_Q1 = train_info2.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（处理异常值前）
    train_cn_Q3 = train_info2.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（处理异常值前）
    train_cn_IQR = train_cn_Q3 - train_cn_Q1
    train_cn_minQ = train_cn_Q1 - (1.5 * train_cn_IQR)    
    Smalloutlier_cn_count = data_train.loc[data_train[cn] < train_cn_minQ ,cn].count() # 训练集特征字段cn超过Q1 - 1.5IQR下界的异常值个数
    Smalloutlier_cn_index = data_train.loc[data_train[cn] < train_cn_minQ, cn].index
    # 生成与特征字段在区间 [Q1 - 1.5IQR, Q1]的随机数，用于填补缺失值
    if (train_cn_Q1 - train_cn_minQ) > 1:
        rand_1min = np.random.randint(low = train_cn_minQ, high = train_cn_Q1+1, size = Smalloutlier_cn_count)    
        rand_dmin = np.random.rand(Smalloutlier_cn_count)    # 生成[0, 1)之间的小数
        rand_imin = rand_1min + rand_dmin    
        data_train.loc[Smalloutlier_cn_index, cn] = rand_imin    # 在异常值的位置填充上生成的随机数
    else:
        data_train.loc[Smalloutlier_cn_index, cn]= train_cn_minQ                 #np.tile(train_cn_minQ, Smalloutlier_cn_count)


In [ ]:
for i in features:
    fig, ax = plt.subplots(figsize = (6, 4))
    sns.boxplot(data = data_train[i])
    ax.set_title('箱形图：' + str(i))
    plt.show()

In [ ]:
test_info = data_test.describe()     # 用于生成训练集的中值，均值，分位数等信息
test_info2 = test_info

In [ ]:
for cn in features:
    test_cn_Q1 = test_info.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（处理异常值前）
    test_cn_Q3 = test_info.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（处理异常值前）
    test_cn_IQR = test_cn_Q3 - test_cn_Q1
    test_cn_maxQ = test_cn_Q3 + (1.5 * test_cn_IQR)

    Largeoutlier_cn_count = data_test.loc[data_test[cn] > test_cn_maxQ ,cn].count() # 训练集特征字段cn超过Q3 + 1.5IQR上界的异常值个数
    Largeoutlier_cn_index = data_test.loc[data_test[cn] > test_cn_maxQ, cn].index
    # 生成与特征字段在区间 [Q3, Q3 + 1.5IQR]的随机数，用于填补缺失值
    if (test_cn_maxQ - test_cn_Q3) > 1:
        rand_1max = np.random.randint(low = test_cn_Q3, high = test_cn_maxQ, size = Largeoutlier_cn_count)    
        rand_dmax = np.random.rand(Largeoutlier_cn_count)    # 生成[0, 1)之间的小数
        rand_imax = rand_1max + rand_dmax    
        data_test.loc[Largeoutlier_cn_index, cn] = rand_imax    # 在异常值的位置填充上生成的随机数
    else:    # 有些特征的值范围非常小，小于1，这样的特征异常值，令其等于test_cn_maxQ
        data_test.loc[Largeoutlier_cn_index, cn] = test_cn_maxQ  

for cn in features:
    test_cn_Q1 = test_info2.loc['25%', cn]     # 训练集特征字段cn的Q1分位数（处理异常值前）
    test_cn_Q3 = test_info2.loc['75%', cn]     # 训练集特征字段cn的Q3分位数（处理异常值前）
    test_cn_IQR = test_cn_Q3 - test_cn_Q1
    test_cn_minQ = test_cn_Q1 - (1.5 * test_cn_IQR)    
    Smalloutlier_cn_count = data_test.loc[data_test[cn] < test_cn_minQ ,cn].count() # 训练集特征字段cn超过Q1 - 1.5IQR下界的异常值个数
    Smalloutlier_cn_index = data_test.loc[data_test[cn] < test_cn_minQ, cn].index
    # 生成与特征字段在区间 [Q1 - 1.5IQR, Q1]的随机数，用于填补缺失值
    if (test_cn_Q1 - test_cn_minQ) > 1:
        rand_1min = np.random.randint(low = test_cn_minQ, high = test_cn_Q1+1, size = Smalloutlier_cn_count)    
        rand_dmin = np.random.rand(Smalloutlier_cn_count)    # 生成[0, 1)之间的小数
        rand_imin = rand_1min + rand_dmin    
        data_test.loc[Smalloutlier_cn_index, cn] = rand_imin    # 在异常值的位置填充上生成的随机数
    else:
        data_test.loc[Smalloutlier_cn_index, cn]= test_cn_minQ              

In [ ]:
for i in features:
    fig, ax = plt.subplots(figsize = (6, 4))
    sns.boxplot(data = data_test[i])
    ax.set_title('箱形图：' + str(i))
    plt.show()

In [ ]:
data_train.shape
data_test.shape

将缺失值处理后的样本集保存到文件

In [ ]:
# 将插值后的数据保存到文件中
data_train.to_csv('data_train_All.csv', encoding = 'gbk')
data_test.to_csv('data_test_All.csv', encoding = 'gbk')

**现在看一下处理完异常值之后的直方图，但是很多特征的直方图分布是近似服从对数正态分布的，如果对其取对数，则特征会服从正态分布，下面分析一下各个特征**  
注：根据后面的结果测试发现，对特征取对数并不能提高测试效果

In [ ]:
from scipy import stats
for i in missing_features:
    fig = plt.figure( figsize = (6, 4))
    sns.distplot(data_train[i], fit = stats.norm)
    plt.show()
    res = stats.probplot(data_train[i], plot = plt)
    print(i)

额，好像处理过异常值之后把原来数据的分布也改变了，，，，

In [ ]:
from scipy import stats
for i in missing_features:
    fig = plt.figure( figsize = (6, 4))
    sns.distplot(np.log1p(data_train[i]), fit = stats.norm)
    plt.show()
    res = stats.probplot(np.log1p(data_train[i]), plot = plt)
    print(i)

**经过对数对数化之后，比对数化之前更接近高斯分布的特征有：**  
\*天门冬氨酸氨基转换酶，\*丙氨酸氨基转换酶, \*碱性磷酸酶，\*r-谷氨酰基转换酶，\*球蛋白，甘油三酯，总胆固醇，高密度脂蛋白胆固醇，低密度脂蛋白胆固醇，尿素，肌酐，尿酸，白细胞计数，血小板体积分布宽度， 嗜酸细胞%，嗜碱细胞%  
**不进行对数变换，比对数变换后更接近高斯分布的特征：**  
\*总蛋白，白蛋白，白球比例，红细胞计数，血红蛋白，红细胞压积，红细胞平均体积，红细胞平均血红蛋白量，红细胞平均血红蛋白浓度，红细胞体积分布宽度，血小板计数，血小板平均体积，血小板比积，中性粒细胞%， 淋巴细胞%， 单核细胞%


In [ ]:
from scipy import stats
for i in missing_features:
    fig = plt.figure( figsize = (6, 4))
    sns.distplot(data_test[i], fit = stats.norm)
    plt.show()
    res = stats.probplot(data_test[i], plot = plt)
    print(i)

In [ ]:
from scipy import stats
for i in missing_features:
    fig = plt.figure( figsize = (6, 4))
    sns.distplot(np.log1p(data_test[i]), fit = stats.norm)
    plt.show()
    res = stats.probplot(np.log1p(data_test[i]), plot = plt)
    print(i)

在测试集中，特征字段的分布和训练集中的特征字段一样，所以对训练集与测试集中相同的特征字段进行对数化或者不进行对数化，在下面进行分析之前，先对上面对数化之后更加近似正态分布的特征进行对数化

In [ ]:
log_features = ['*天门冬氨酸氨基转换酶', '*丙氨酸氨基转换酶', '*碱性磷酸酶', '*r-谷氨酰基转换酶', '*球蛋白',
                '甘油三酯', '总胆固醇',  '高密度脂蛋白胆固醇', '低密度脂蛋白胆固醇', '尿素', '肌酐', '尿酸',
                '白细胞计数', '血小板体积分布宽度', '嗜酸细胞%', '嗜碱细胞%']
data_train[log_features] = np.log1p(data_train[log_features])
data_test[log_features] = np.log1p(data_test[log_features])

下面这个函数的功能是划分数据集，对数据进行预处理

In [ ]:
# 将原来的训练集分为训练集和交叉验证集
from sklearn.preprocessing import PolynomialFeatures
def train_seperate_PFS(train, test, size = 0.2, scale = 'NoScale', poly_features = False):
    """
    参数：
    size是划分训练集和验证集的比例
    scale是归一化方式
    返回：
    x_train是从所有训练集中划分得到的训练集， label_train是其标签
    x_dev是从所有训练集中划分得到的验证集， label_dev是其标签
    X_train是全部的训练集， Y_train是其标签
    Test是全部的训练集
    """        
    features_train = train.loc[:, train.columns != '血糖']     # 全部的训练集特征
    labels = train.loc[:, train.columns == '血糖']             # 全部的训练集标签
    Test = test               # 全部的测试集
    if poly_features == True:
        poly = PolynomialFeatures(2, interaction_only = True)     # 先生成多项式特征，然后决定是否进行归一化       
        features_train = poly.fit_transform(features_train)
        Test = poly.fit_transform(Test)

    if scale == 'Standard':    # 对数据进行均值归一化
        scaler = StandardScaler().fit(features_train)
        features_train = scaler.transform(features_train)
        Test = scaler.transform(Test)       
    if scale == 'Robust':     # 对数据进行robust_scale
        robust_scaler = RobustScaler().fit(features_train)
        features_train = robust_scaler.transform(features_train)
        Test =  robust_scaler.transform(Test)
    if scale == 'NoScale':        # 不进行归一化
        features_train = features_train
        Test = Test
        

    # 将原来训练集中的20%的样本分为交叉验证集
    state = np.random.seed(123)
    x_train, x_dev, label_train, label_dev = train_test_split(features_train, labels, test_size = size, random_state = state) 
        
    print('训练集的样本个数是：' + str(x_train.shape[0]))
    print('训练集的形状是：' + str(x_train.shape))
    print('交叉验证集的样本个数是：' + str(x_dev.shape[0]))
    print('所有训练集的样本个数是：' + str(features_train.shape[0]))
    print('测试集的样本个数是：' + str(Test.shape[0]))
    return x_train, x_dev, label_train, label_dev, features_train, labels, Test

In [ ]:
x_train, x_dev, label_train, label_dev, X_train, Y_train, X_test = train_seperate_PFS(data_train,
                                                                                    data_test,
                                                                                    size = 0.2,
                                                                                    scale = 'NoScale',
                                                                                    poly_features = False)    # 将训练集样本划分为训练集和交叉验证集

### XGBoost

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
Y_test = pd.read_csv('d_answer_a_20180128.csv', header = None)
Y_test.shape

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 1000, 
                             max_depth = 6,
                             learning_rate = 0.01,
                             gamma = 40,
                             subsample = 0.8,
                             min_child_weight = 1,
                             colsample_bytree = 0.7)
xgb_model.fit(X_train, Y_train.values.ravel())
y_train_pred = xgb_model.predict(X_train)
y_dev_pred = xgb_model.predict(X_test)
loss1 = mean_squared_error(y_train_pred, Y_train) * 0.5
loss1
loss2 = mean_squared_error(y_dev_pred, Y_test) * 0.5
loss2

In [ ]:
xr_train, xr_dev, labelr_train, labelr_dev, Xr_train, Yr_train, Xr_test = train_seperate_PFS(data_train,
                                                                                    data_test,
                                                                                    size = 0.2,
                                                                                    scale = 'NoScale',
                                                                                    poly_features = False)    # 将训练集样本划分为训练集和交叉验证集

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators = 1000, 
                             max_depth = 6,
                             learning_rate = 0.05,
                             gamma = 40,
                             subsample = 0.8,
                             min_child_weight = 1,
                             colsample_bytree = 0.7)
xgb_model.fit(Xr_train, Yr_train.values.ravel())
y_train_pred = xgb_model.predict(Xr_train)
y_dev_pred = xgb_model.predict(Xr_test)
loss1 = mean_squared_error(y_train_pred, Yr_train) * 0.5
loss1
loss2 = mean_squared_error(y_dev_pred, Y_test) * 0.5
loss2

将训练集中的部分特征取对数后，生成平方多项式的效果会好些，但是效果不如原始数据  
原始数据中的异常值处理之后，效果稍微比没处理的时候稍微差一点

下面试一下5折交叉验证

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = np.random.seed(123))   # 5折交叉验证
Xr_train = np.array(Xr_train)
Yr_train = np.array(Yr_train)

for k, (train_index, test_index) in enumerate(kfold.split(Xr_train, Yr_train)):
    xgb_model = xgb.XGBRegressor(n_estimators = 1000, 
                             max_depth = 6,
                             learning_rate = 0.05,
                             gamma = 40,
                             subsample = 0.8,
                             min_child_weight = 1,
                             colsample_bytree = 0.7)
    
    print('第 {} 次训练.....'.format(k))
    
    xgb_model.fit(Xr_train[train_index], Yr_train[train_index])
    y_tr_pred = xgb_model.predict(Xr_train[train_index])
    y_de_pred = xgb_model.predict(Xr_train[test_index])
    loss1 = mean_squared_error(y_tr_pred, Yr_train[train_index]) * 0.5
    print("训练集误差：" + str(loss1))
    loss2 = mean_squared_error(y_de_pred, Yr_train[test_index]) * 0.5
    print("验证集误差：" + str(loss2))